In [4]:
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from future.utils import iteritems
from collections import Counter
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pymysql
from gensim.models import word2vec
from tqdm.notebook import tqdm

import sys
if sys.version_info <= (2,7):
    reload(sys)
    sys.setdefaultencoding('utf-8')
import konlpy
from konlpy.tag import Kkma, Okt, Hannanum

In [5]:
article_data = pd.read_csv('mixver9.txt', encoding='utf-8', header= None)
documents = [' '.join(i[0].split(' ')[1:]) for i in article_data.values]

as_one = ''
for document in documents:
    as_one = as_one + ' ' + document
words = as_one.split()

counts = Counter(words)

vocab = sorted(counts, key=counts.get, reverse=True)

word2idx = {word.encode("utf8").decode("utf8"): ii for ii, word in enumerate(vocab,1)}

idx2word = {ii: word for ii, word in enumerate(vocab)}

V = len(word2idx)
N = len(documents)

tf = CountVectorizer()

tf.fit_transform(documents)

tf.fit_transform(documents)[0:1].toarray()

tfidf = TfidfVectorizer(max_features = 1000, max_df=1, min_df=0)

#generate tf-idf term-document matrix
A_tfidf_sp = tfidf.fit_transform(documents)  #size D x V

tfidf_dict = tfidf.get_feature_names()

In [6]:
kkma = Kkma()
okt = Okt()
hannanum = Hannanum()
print('konlpy version = %s' % konlpy.__version__)

konlpy version = 0.5.2


In [42]:
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password='0000',db='test', charset='utf8')
curs = conn.cursor()

project_title = '꿈을 담다. 꿈 해몽 카드 모르몽'
project_category ='디자인'

tokenized_project_title = set(okt.nouns(project_title))
selected_word=[]

for i in tokenized_project_title:
    if i in tfidf_dict:
        selected_word.append(i)
        continue
model = word2vec.Word2Vec.load("NaverMovie29.model")

print(selected_word)

t_cnt =0
w_cnt = 0

for i in selected_word:
    similar_word=model.wv.most_similar(positive=[i],topn=5)
    print(similar_word)
    for j in similar_word:
        sql = 'select pagename from test.crawl where category="%s" and title like "%%%s%%" and achieve>100;'%(project_category,j[0])
        curs.execute(sql)
        pagename = curs.fetchall()
        print(pagename)
        print(type(pagename))
        for k in pagename:
            if k[0] == 'tumblbug':
                t_cnt+=1
            elif k[-]:
                
                w_cnt+=1
        

print(t_cnt, w_cnt)
if t_cnt > w_cnt:
    print("tumblbug")
elif t_cnt == w_cnt:
    print("same")
else:
    print("wadiz")
conn.close()

SyntaxError: invalid syntax (<ipython-input-42-54d92403bdc6>, line 33)

In [ ]:
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password='0000',db='test', charset='utf8')

curs = conn.cursor()


#print('꼬꼬마 명사: ', kkma.nouns(sent))
#print(okt.nouns(sent))
#print('한나눔 명사: ', hannanum.nouns(sent))
username="박가영"
sql = "select distinct trim(I.title) from test.user_info I, test.crawl C where username like '%s' and C.title = I.title;"%username
#sql1 = "select distinct I.title, C.category from test.user_info I, test.crawl C where username like '%s' and C.title = I.title;"%username

curs.execute(sql)
sent = curs.fetchall()

"""
curs.execute(sql1)
title_with_category = curs.fetchall()
print("=======title with category=========")
#print(title_with_category)
#for i in title_with_category:
#    print(i[1])
print("=======title with category=========")
"""
noun_set = set(okt.nouns(str(sent)))
selected_word = []
#print(sent)
sent_list = list(sent)
sent_list2 = []
for i in sent_list:
    sent_list2.append(list(i))
#print(sent_list2)
#selected_word : tokenized word from user's funding title list
for i in noun_set:
    if i in tfidf_dict:
        selected_word.append(i)
        continue
final_word_list = []
category_list=[]
#final_word_list : selected words
print("=======category list=======")
for i in selected_word:
    sql3 = "select distinct I.title, C.category from test.user_info I, test.crawl C where username like '{username}' and C.title = I.title and I.title like '%%{word}%%';".format(username=username, word=i);
    curs.execute(sql3)
    rows = curs.fetchall()
    #print(rows)
    
    for j in rows:
        if i in j[0]:
            final_word_list.append(i)
            category_list.append(j[1])
            #print(i, j[1])
print("=======category list=======")  

model = word2vec.Word2Vec.load("NaverMovie29.model")
word_len = len(final_word_list)
category_len = len(category_list)

final_projects=[]
similar_word_list=[]
for i in tqdm(range(word_len)):
    print("<<key word: "+final_word_list[i], category_list[i]+">>")
    print()
    similar_word=model.wv.most_similar(positive=[final_word_list[i]],topn=5)
    for j in similar_word:
        if len(j[0])==1:
            continue
        else:
            print(j[0])
            sql = "select DISTINCT pagename, category, trim(title) from test.crawl where title like '%%%s%%'" %j[0]
            curs.execute(sql)
            words = curs.fetchall()
            conn.commit()
            #print(words)
            length = len(words)
            if length>3:
                length = 3
            #print(length)
            cnt1=0
            cnt2=0
            for k in range(0,length):
                tmp=[]
                tmp.append(''.join(list(words[k][2])))
                if words[k][1] == category_list[i] and tmp not in sent_list2 :
                    print('추천할 수 있는 프로젝트 ', end='')
                    cnt1+=1
                    print(cnt1)
                    final_projects.append(words[k])

        
        print()   


final_projects_set = set(final_projects)
for i in final_projects_set:
    print(i)
print(len(final_projects_set))
conn.close()